### **API SADVR - Portrait statistique**  
https://www.cen.umontreal.ca/espacedoc/sadvr/  

Ce NoteBook est destiné à l'extraction et la visualisation de statistiques relatives aux professeur·e·s et à leurs expertises à partir de l'API de la vitrine de la recherche (SADVR). 
Celles-ci seront intégrées en un tableau de bord [PowerBI](https://wiki.umontreal.ca/display/SIE/Power+BI) permettant d'avoir un portrait d'ensemble des données.  

---

In [ ]:
import pandas as pd
from SADVR_utils import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from rdflib import *

**Chargement des données**

In [ ]:
data = updateInfoProfs()

**Expertises de recherche**

- Affiliations (facultés, départements, établissements affiliés)
- Disciplines
- Secteur de recherche
- Périodes chronologiques étudiées
- Régions géographiques / pays étudiés

In [ ]:
expertises = data[['idsadvr', 'affiliations', 'etablissementsAffilies', 'expertise']]

toNormalize = ['affiliations', 'etablissementsAffilies', 'expertise', 'expertise.secteursRecherche',
                'expertise.disciplines', 'expertise.pays', 'expertise.continents', 'expertise.periodesChronologiques']

for c in toNormalize:
    expertises = explodeNormalize(expertises, c)

drop = ['affiliations.courrielInstitutionnel', 'affiliations.immeuble', 'affiliations.fonction.codeSad', 
        'affiliations.fonction.nom', 'affiliations.local', 'affiliations.exclusion', 'affiliations.exclusionTel',
        'affiliations.uniteAdministrative.codeSad', 'affiliations.uniteAdministrative.nom', 'affiliations.telephone.numero',
        'affiliations.telephone.poste', 'expertise.phraseCle']

expertises = expertises.drop(columns=drop)

*Établissements affiliés*

In [ ]:
# Etablissements affiliés
etablissementsAffilies = expertises.dropna(subset='etablissementsAffilies.nom')
etablissementsAffilies = etablissementsAffilies.drop_duplicates(subset=(['idsadvr', 'etablissementsAffilies.nom']))
etablissementsAffilies = pd.DataFrame(plotVariable(etablissementsAffilies, 'etablissementsAffilies.nom')).sort_values(by='count')

etablissementsAffilies

In [ ]:
# px.bar(
#     y = etablissementsAffilies['labels'],
#     x = etablissementsAffilies['count'],
#     orientation = 'h',
#     height = 800
# )

**Facultés, départements et établissements affiliés**

*Nombre de professeurs par facultés*

In [ ]:
facultes = pd.DataFrame(plotVariable(expertises, 'affiliations.faculte.nom'))[:-2].sort_values(by='count')
facultes

In [ ]:
px.bar(
    facultes, 
    x= facultes['count'], 
    y= facultes['labels'], 
    title='Nombre de professeur-e-s par facultés',
    width=1000, 
)

*Secteurs de recherche*

In [ ]:
secteursRecherche = pd.DataFrame(plotVariable(filtrerLangue(expertises), 'expertise.secteursRecherche.nom')).sort_values(by='count')

secteursRecherche

In [ ]:
px.bar(
    x = secteursRecherche['count'],
    y = secteursRecherche['labels'],
    orientation = 'h',
    height = 1000
)

**Expertises de recherche: cartographie des expertises par mots-clés**

- Mots-clés / Phases clés associés aux disciplines / facultés / départements de recherche 

Voir si on peut extraire un graphe et le visualiser

*Disciplines*

In [32]:
disciplines = expertises[[x for x in expertises.columns if 'discipline' in x or 'motsCles' in x or 'phraseCle' in x or x == 'idsadvr']]
disciplines

,idsadvr,expertise.motsCles,expertise.disciplines.uid,expertise.disciplines.codeLangue,expertise.disciplines.nom,expertise.disciplines.ordre
0,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",19,fre,Biochimie,2
1,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",20,fre,Biologie cellulaire,1
2,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",22,fre,Biologie moléculaire,3
3,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",19,eng,Biochemistry,2
4,in13580,"[{'uid': '5', 'nom': 'Acides nucléiques', 'ord...",20,eng,Cell Biology,1
...,...,...,...,...,...,...
285318,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",77,eng,Computer Science,1
285319,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",78,eng,Administrative Computing,2
285320,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",83,eng,Management,3
285321,in35999,"[{'uid': '2777', 'nom': 'Sécurité informatique...",86,eng,Pure Mathematics,4


In [33]:
mappingDisciplines = pd.read_csv('tables/SADVR_disciplines.csv')
mappingDisciplines = mappingDisciplines[mappingDisciplines['noms.codeLangue'] == 'fre']
mappingDisciplines = {str(x['id']): x['noms.nom'] for x in mappingDisciplines.to_dict('records')}
freqDisciplines = pd.DataFrame(plotVariable(disciplines, 'expertise.disciplines.uid', mapping=mappingDisciplines))
freqDisciplines

,expertise.disciplines.uid,count,mapping
0,98,196,Neurosciences
1,133,194,Sociologie
2,121,180,Psychologie
3,22,179,Biologie moléculaire
4,155,167,Santé publique
...,...,...,...
201,200,1,Oncologie gynécologique
202,65,1,Génie industriel
203,194,1,Mycologie
204,219,1,Anatomie


In [35]:
# 1 premier exemple: Neurosciences
# Une fois que ça aura fonctionné pour une discipline, on va créer une fonction qui pourra le faire pour toutes les disciplines
neuro = disciplines[disciplines['expertise.disciplines.uid'] == '98']
neuro = explodeNormalize(neuro, 'expertise.motsCles')
neuro = neuro.dropna(subset='expertise.motsCles.nom')
neuro = neuro[neuro['expertise.motsCles.ordre'].astype(int) <= 5]

neuro = neuro.drop_duplicates(subset=['idsadvr', 'expertise.motsCles.uid'])
neuro = neuro [['idsadvr',	'expertise.disciplines.uid', 'expertise.disciplines.nom', 'expertise.motsCles.uid',
                'expertise.motsCles.nom',	'expertise.motsCles.ordre',	'expertise.motsCles.codeLangue']]

freqMotsCles = plotVariable(neuro, 'expertise.motsCles.uid').to_dict('records')
freqMotsCles = {x['expertise.motsCles.uid'] : x['count'] for x in freqMotsCles}

neuro['expertises.motCles.count'] = neuro['expertise.motsCles.uid'].map(freqMotsCles)
neuro = neuro.drop_duplicates(subset = ['expertise.motsCles.uid']).drop(columns='idsadvr')

neuro = neuro.sort_values(by='expertises.motCles.count', ascending=False).reset_index(drop=True)
neuro

,expertise.disciplines.uid,expertise.disciplines.nom,expertise.motsCles.uid,expertise.motsCles.nom,expertise.motsCles.ordre,expertise.motsCles.codeLangue,expertises.motCles.count
0,98,Neurosciences,98,Communication neuronale et neurotransmission,2,fre,25
1,98,Neurosciences,95,Cognition,1,fre,17
2,98,Neurosciences,5386,Neuroscience cognitive,2,fre,16
3,98,Neurosciences,1,Activité locomotrice / troubles du mouvement,1,fre,12
4,98,Neurosciences,5348,Imagerie cérébrale,3,fre,12
...,...,...,...,...,...,...,...
349,98,Neurosciences,505,Syntaxe,4,fre,1
350,98,Neurosciences,575,Troubles du langage chez l'enfant,5,fre,1
351,98,Neurosciences,204,Famille et aidants naturels,3,fre,1
352,98,Neurosciences,199,Évolution et phylogénie,3,fre,1


*On va essayer de créer un graphe avec*

In [36]:
graph = neuro[:30]
listeMotsCles = graph.to_dict('records')

idDiscipline = str(98)

uriDisciplines = URIRef(f'{baseURI}/disciplines')
uriDiscipline = URIRef(f'{baseURI}/disciplines/{idDiscipline}')
uriCount = URIRef(f'{baseURI}/count')
uriHasKW = URIRef(f'{baseURI}/hasKeyword')

g = Graph()
g.bind('sadvr', 'https://www.recherche.umontreal.ca/vitrine/rest/api/1.7/umontreal/')

# Classe pour les disciplines
g.add((uriDisciplines, RDF.type, OWL.Class))
g.add((uriDisciplines, RDFS.label, Literal('Discipline', lang='fr')))

# Classe et nom de la discipline (fr)
g.add((uriDiscipline, RDF.type, uriDisciplines))
g.add((uriDiscipline, RDF.type, OWL.Class))
g.add((uriDiscipline, RDFS.label, Literal(mappingDisciplines[idDiscipline], lang='fr')))

# Relations
# keyWordCount
g.add((uriCount, RDF.type, OWL.ObjectProperty))
g.add((uriCount, RDFS.label, Literal('keywordCount', lang='en')))

for kw in listeMotsCles:
    id = kw['expertise.motsCles.uid']
    uri = URIRef(f'{baseURI}/motsCles/{id}')
    label = Literal(kw['expertise.motsCles.nom'])
    compte = Literal(kw['expertises.motCles.count'], datatype=XSD.integer)

    # Lien discipline <> mot-clé
    g.add((uri, RDFS.subClassOf, uriDiscipline))

    # Construction du noeud pour le mot-clé
    g.add((uri, RDF.type, OWL.Class))
    g.add((uri, RDFS.label, label))
    g.add((uri, uriCount, compte))

[{'expertise.disciplines.uid': '98',
  'expertise.disciplines.nom': 'Neurosciences',
  'expertise.motsCles.uid': '98',
  'expertise.motsCles.nom': 'Communication neuronale et neurotransmission',
  'expertise.motsCles.ordre': '2',
  'expertise.motsCles.codeLangue': 'fre',
  'expertises.motCles.count': 25},
 {'expertise.disciplines.uid': '98',
  'expertise.disciplines.nom': 'Neurosciences',
  'expertise.motsCles.uid': '95',
  'expertise.motsCles.nom': 'Cognition',
  'expertise.motsCles.ordre': '1',
  'expertise.motsCles.codeLangue': 'fre',
  'expertises.motCles.count': 17},
 {'expertise.disciplines.uid': '98',
  'expertise.disciplines.nom': 'Neurosciences',
  'expertise.motsCles.uid': '5386',
  'expertise.motsCles.nom': 'Neuroscience cognitive',
  'expertise.motsCles.ordre': '2',
  'expertise.motsCles.codeLangue': 'fre',
  'expertises.motCles.count': 16},
 {'expertise.disciplines.uid': '98',
  'expertise.disciplines.nom': 'Neurosciences',
  'expertise.motsCles.uid': '1',
  'expertise.mot

In [68]:
g.serialize('visualisations/graph_test_neurosciences.ttl', format='ttl')

<Graph identifier=N302e4107e7c140c3a513fae45cab9d85 (<class 'rdflib.graph.Graph'>)>

**Rayonnement académique**

- Publications (Nombre de publications par Faculté - à travers le temps ? - format des notices peu standard)